## Try running vanilla MD on ala dipeptide in solvent at the first thermostate and see if the problem happens

In [1]:
from tqdm import tqdm_notebook
import openmmtools

In [2]:
import pickle
import os
from openeye import oechem
from perses.annihilation.rest import RESTTopologyFactoryV2
from perses.annihilation.lambda_protocol import RESTStateV2
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.testsystems import AlanineDipeptideExplicit


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.3 jupyter extensions


In [3]:
length = 1
move_length = 1
timestep = 4
radius = 0.4


In [6]:
# # Load rhtf
# htf = pickle.load(open("T42A_complex_0.pickle", "rb" ))



In [8]:
# # Build REST factory
# for res in htf.hybrid_topology.residues:
#     if res.resSeq == 83 and res.chain.index == 0:
#         mutated_res = res
# query_indices = [atom.index for atom in mutated_res.atoms]
# traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
# solute_atoms = list(traj.topology.select("is_protein"))
# rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
# rest_atoms = [int(atom) for atom in rest_atoms]
# factory = RESTTopologyFactoryV3(htf.hybrid_system, htf.hybrid_topology, rest_region=rest_atoms, use_dispersion_correction=True)

INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 2.7569734238004693
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]


In [4]:
## CASE 2: alanine dipeptide in solvent
# Create vanilla system
ala = AlanineDipeptideExplicit()
system = ala.system
system.removeForce(4)
positions = ala.positions
topology = ala.topology

# Create REST system
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = RESTTopologyFactoryV2(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
REST_system = factory.REST_system


INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:REST:Handling constraints
INFO:REST:Handling bonds
INFO:REST:Handling angles
INFO:REST:Handling torsions
INFO:REST:Handling nonbondeds
INFO:REST:Handling nonbonded scaling (custom nb)
INFO:REST:Handling nonbonded exception scaling (custom bond)


In [6]:
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 300.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

# Create thermodynamics states
sampler_state =  SamplerState(positions, box_vectors=system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in [temperatures[0], temperatures[-1]]:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state, max_iterations=0)
    sampler_state_list.append(copy.deepcopy(sampler_state))


In [7]:
REST_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad543ee8930> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ad543ef4510> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ad543ef4600> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ad543ef4060> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad543ef4030> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad5c1db8690> >]

In [8]:
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin,
                                              collision_rate=1 / unit.picoseconds,
                                              timestep= 4 * unit.femtoseconds)
context_cache = cache.global_context_cache
context, integrator = context_cache.get_context(thermodynamic_state_list[0], integrator)


In [9]:
sampler_state_list[0].apply_to_context(context)

In [10]:
old, new = list(), list()
for _ in tqdm_notebook(range(int(125000/250))):
    integrator.step(250)
    
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
#     old_pos = np.asarray(htf.old_positions(pos))
#     new_pos = np.asarray(htf.new_positions(pos))
#     old.append(old_pos)
#     new.append(new_pos)
    old.append(pos)

<ipython-input-10-315e5c042bd3>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(int(125000/250))):


  0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
with open("ala_300K_v2.npy", 'wb') as f:
    np.save(f, old)
# with open("new_1200K.npy", 'wb') as f:
#     np.save(f, new)

In [12]:
import mdtraj as md

In [13]:
traj = md.Trajectory(np.array(old), md.Topology.from_openmm(ala.topology))
traj.save("ala_300K_v2.dcd")
traj[0].save("ala_300K_v2.pdb")